In [31]:
import numpy as np
import pandas as pd

In [7]:
4*.015/(np.pi*.1)

0.1909859317102744

In [368]:
class Winslow(object):
    def __init__(self, rotorradius=.1, bladenumber=4, solidity=.2):
        self.rotorradius = rotorradius
        self.bladenumber = bladenumber
        self.solidity = solidity
        
    def rotormass(self):
        return 0.0195*self.rotorradius**(2.0859)*self.solidity**(-.2038)*self.bladenumber**(.5344)
    
    def batterymass(self, capacity, seriesnumber):
        return .0418*capacity**(.9327)*seriesnumber**(1.0725)
    
    def motormassBL(self, Kv, P, I):
        lBL = 4.8910*I**(.1751)*P**(.2476)
        dBL = 41.45*Kv**(-.1919)*P**(.1935)
        return .0109*Kv**(.5122)*P**(-.1902)*np.log10(lBL)**(2.5582)*np.log10(dBL)**(12.8502)
    
    def motormassDC(self, P, Qmax):
        lDC = 20.83*Qmax**(.1924)
        dDC = 11.13*Qmax**(.2895)
        return (10**(-81))*P**(-.2979)*Qmax**(-20.5615)*np.log10(lDC)**(746)*np.log10(dDC)**(-212)
    
    def ESCmassBL(self, Imax):
        return 0.8013*Imax**(.9727)
    
    def ESCmassDC(self, Imax):
        return .977*Imax**(.8483)
    
    def airframemass(self, R, capacity, seriesnumber):
        return 1.3119*R**(1.2767)*self.batterymass(capacity, seriesnumber)**(.4587)
            
    @property
    def rotorradius(self):
        return self._rotorradius
    
    @rotorradius.setter
    def rotorradius(self, rr):
        try:
            rr = np.abs(float(rr))
        except:
            pass
        else:
            self._rotorradius = rr*1e2
            
    @property
    def bladenumber(self):
        return self._bladenumber
    
    @bladenumber.setter
    def bladenumber(self, bn):
        try:
            bn = np.abs(int(bn))
        except:
            pass
        else:
            self._bladenumber = bn
            
    @property
    def solidity(self):
        return self._solidity
    
    @solidity.setter
    def solidity(self, sol):
        try:
            sol = np.abs(float(sol))
        except:
            pass
        else:
            self._solidity = sol

In [67]:
T_prop

3.4671743578615275

In [66]:
W_prop

45.54663953378007

In [369]:
uav = Winslow()

Source: https://sunnyskyusa.com/collections/x-motors/products/sunnsky-x2212

In [59]:
tableAPC8038 = """
0.7	100	10.36	9.65	4120
1.7	200	25.16	7.95	5641
2.8	300	41.44	7.24	6668
4	400	59.2	6.76	7629
5.4	500	79.92	6.26	8440
7.1	600	105.08	5.71	9154
8.7	700	128.76	5.44	9766
10.2	800	150.96	5.30	10436
16.4	1075	242.72	4.43	11910
"""

tableAPC9045_11 = """
0.9	100	9.99	10.01	3406
2.1	200	23.31	8.58	4827
3.3	300	36.63	8.19	5789
4.8	400	53.28	7.51	6642
6.5	500	72.15	6.93	7370
12.2	785	135.42	5.80	9308
"""

tableAPC9045_15 = """
0.6	100	8.88	11.26	3452
1.5	200	22.2	9.01	4734
2.7	300	39.96	7.51	5780
3.8	400	56.24	7.11	667
5.2	500	76.96	6.50	746
6.6	600	97.68	6.14	8140
8.4	700	124.32	5.63	8649
10	800	148	5.41	9150
11.9	900	176.12	5.11	9696
14	1000	207.2	4.83	11368
16	1100	236.8	4.65	10221
18.2	1220	269.36	4.53	10649
"""

tableAPC9047_11 = """
0.8	100	8.88	11.26	3288
1.9	200	21.09	9.48	4580
3.2	300	35.52	8.45	5638
4.9	400	54.39	7.35	6591
6.4	500	71.04	7.04	7301
8.2	600	91.02	6.59	8041
10.4	700	115.44	6.06	9396
13.2	840	146.52	5.73	8730
"""

tableAPC9047_15 = """
0.6	100	8.88	11.26	3231
1.5	200	22.2	9.01	4643
2.5	300	37	8.11	5605
3.8	400	56.24	7.11	6432
5	500	74	6.76	7378
6.5	600	96.2	6.24	8090
8.2	700	121.36	5.77	8730
10	800	148	5.41	9366
11.7	900	173.16	5.20	10131
13.8	1000	204.24	4.90	10716
18.4	1250	272.32	4.59	11755
"""

tableAPC1038 = """
0.7	100	7.77	12.87	2813
1.8	200	19.98	10.01	3866
3	300	33.3	9.01	4636
4.6	400	51.06	7.83	5170
6.3	500	69.93	7.15	5687
8.1	600	89.91	6.67	6100
10.2	700	113.22	6.18	6591
12.4	800	137.64	5.81	6853
15.3	900	169.83	5.30	7262
20.2	1130	224.22	5.04	7923
"""

tableAPC1047_11 = """
0.7	100	7.77	12.87	2612
1.7	200	18.87	10.60	3605
3	300	33.3	9.01	4264
4.5	400	49.95	8.01	4887
6.4	500	71.04	7.04	5418
8.3	600	92.13	6.51	5872
10.45	700	115.99	6.03	6335
12.7	800	5	5.67	6983
15.3	900	140.97	5.30	7378
19.9	1100	169.83	4.98	7748
"""

tableAPC1047_15 = """
0.5	100	220.89	13.51	2600
1.3	200	7.4	10.40	3593
2.2	300	19.24	9.21	4253
3.4	400	32.56	7.95	4869
4.9	500	50.32	6.89	5381
6.5	600	72.52	6.24	5845
8.7	700	96.2	5.44	6332
10.5	800	128.76	5.15	6610
12.4	900	155.4	4.90	7055
14.7	1000	183.52	4.60	7444
16.4	1100	217.56	4.53	7766
18	1200	242.72	4.50	8100
20.6	1300	266.4	4.26	8394
24.5	1400	304.88	3.86	8620
27.4	1500	362.6	3.70	8900
29	1650	405.52	3.84	9409
"""

def table2DF(table):
    table = np.asarray([[float(subval) for subval in val.split("\t")] for val in table.split('\n') if val != ""])
    df = pd.DataFrame({
        "Amps" : table[:,0],
        "Thrust" : table[:,1]*9.80665*1e-3,
        "Watts" : table[:,2],
        "Efficiency" : table[:,3],
        "RPM" : table[:,4],
    })
    
    return df

tableAPC8038 = table2DF(tableAPC8038)
tableAPC9045_11 = table2DF(tableAPC9045_11)
tableAPC9045_15 = table2DF(tableAPC9045_15)
tableAPC9047_11 = table2DF(tableAPC9047_11)
tableAPC9047_15 = table2DF(tableAPC9047_15)
tableAPC1038 = table2DF(tableAPC1038)
tableAPC1047_11 = table2DF(tableAPC1047_11)
tableAPC1047_15 = table2DF(tableAPC1047_15)

In [76]:
X2212V3KV980 = {
    11.1: {
        "APC9045" : tableAPC9045_11,
        "APC9047" : tableAPC9047_11,
        "APC1038" : tableAPC1038,
        "APC1047" : tableAPC1047_11
    },
    14.8: {
        "APC8038" : tableAPC8038,
        "APC9045" : tableAPC9045_15,
        "APC9047" : tableAPC9047_15,
        "APC1047" : tableAPC1047_15
    }
}

In [489]:
tableAPC9045 = """
0.9	100	9.99	10.01	3451
2.1	200	23.31	8.58	4722
3.6	300	39.96	7.51	5767
5.1	400	56.61	7.07	6665
6.9	500	76.59	6.53	7409
9	600	99.9	6.01	8035
11	700	122.1	5.73	8581
13.5	800	149.85	5.34	9286
16.2	900	179.82	5.01	9690
19.2	1000	213.12	4.69	10170
25.5	1210	283.05	4.27	11234
"""

tableAPC9047 = """
0.8	100	8.88	11.26	3215
2	200	22.2	9.01	4595
3.3	300	36.63	8.19	5659
5.1	400	56.61	7.07	6569
6.8	500	75.48	6.62	7337
8.8	600	97.68	6.14	8041
10.9	700	120.99	5.79	8934
13.5	800	149.85	5.34	9510
16	900	177.6	5.07	10102
18.9	1000	209.79	4.77	10765
23.6	1220	261.96	4.66	11475
"""

tableAPC9060 = """
0.9	100	9.99	10.01	3185
2.2	200	24.42	8.19	4414
3.9	300	43.29	6.93	5287
5.7	400	63.27	6.32	6129
7.9	500	87.69	5.70	6765
10.1	600	112.11	5.35	7390
12.5	700	138.75	5.05	7881
15	800	166.5	4.80	8393
18.3	900	203.13	4.43	8990
22	1000	244.2	4.10	9402
25.5	1100	283.05	3.89	9887
31.2	1290	346.32	3.72	10547
"""

tableAPC8060_11 = """
1.2	100	13.32	7.51	3829
3	200	33.3	6.01	5533
5.1	300	56.61	5.30	6544
7.1	400	78.81	5.08	7561
9.2	500	102.12	4.90	8315
12	600	133.2	4.50	9094
15.6	700	173.16	4.04	9840
18.7	800	207.57	3.85	10403
24	950	266.4	3.57	11277
"""

tableAPC8060_15 = """
0.9	100	13.32	7.51	3897
2.2	200	32.56	6.14	5490
3.7	300	54.76	5.48	6622
5.4	400	79.92	5.01	7560
7.1	500	105.08	4.76	8484
9.1	600	134.68	4.46	9134
12.1	700	179.08	3.91	9908
15.15	800	224.22	3.57	10486
18.3	900	270.84	3.32	11119
22.5	1000	333	3.00	11586
25.4	1100	375.92	2.93	12147
29.3	1200	433.64	2.77	12573
34.8	1350	515.04	2.62	13427
"""

def table2DF(table):
    table = np.asarray([[float(subval) for subval in val.split("\t")] for val in table.split('\n') if val != ""])
    df = pd.DataFrame({
        "Amps" : table[:,0],
        "Thrust" : table[:,1]*9.80665*1e-3,
        "Watts" : table[:,2],
        "Efficiency" : table[:,3],
        "RPM" : table[:,4],
    })
    
    return df

tableAPC9045 = table2DF(tableAPC9045)
tableAPC9047 = table2DF(tableAPC9047)
tableAPC9060 = table2DF(tableAPC9060)
tableAPC8060_11 = table2DF(tableAPC8060_11)
tableAPC8060_15 = table2DF(tableAPC8060_15)

X2212V3KV1250 = {
    11.1: {
        "APC9045" : tableAPC9045,
        "APC9047" : tableAPC9047,
        "APC9060" : tableAPC9060,
        "APC8060" : tableAPC8060_11
    },
    14.8: {
        "APC8060" : tableAPC8060_15
    }
}

In [490]:
tableAPC9045 = """
1	100	11.1	9.01	3340
2.2	200	24.42	8.19	4705
3.8	300	42.18	7.11	5795
5.5	400	61.05	6.55	6688
7.3	500	81.03	6.17	7419
9.4	600	104.34	5.75	8060
11.5	700	127.65	5.48	8774
14	800	155.4	5.15	9264
17	900	188.7	4.77	9796
19.8	1000	219.78	4.55	10286
22.8	1100	253.08	4.35	10710
26.4	1200	293.04	4.10	11152
29.7	1300	329.67	3.94	11600
31.8	1440	352.98	4.08	12100
"""

tableAPC9047 = """
1	100	11.1	9.01	3244
2.1	200	23.31	8.58	4590
3.6	300	39.96	7.51	5651
5.4	400	59.94	6.67	6509
7.3	500	81.03	6.17	7383
9.3	600	103.23	5.81	8061
11.3	700	125.43	5.58	8839
13.4	800	148.74	5.38	9476
16.1	900	178.71	5.04	10002
18.8	1000	208.68	4.79	10550
21.4	1100	237.54	4.63	11250
24.8	1200	275.28	4.36	11700
30.5	1380	338.55	4.08	12423
"""

tableAPC8038 = """
1	100	11.1	9.01	4002
2.5	200	27.75	7.21	5595
4.1	300	45.51	6.59	6683
5.8	400	64.38	6.21	7620
7.6	500	84.36	5.93	8404
9.9	600	109.89	5.46	9145
12.3	700	136.53	5.13	9798
15	800	166.5	4.80	10389
18	900	199.8	4.50	10992
20.4	1000	226.44	4.42	11503
23.8	1100	264.18	4.16	11963
28	1230	310.8	3.96	12455
"""

tableAPC8060 = """
1.2	100	13.32	7.51	3890
2.9	200	32.19	6.21	5456
5	300	55.5	5.41	6606
7.2	400	79.92	5.01	7564
9.3	500	103.23	4.84	8340
12	600	133.2	4.50	9142
15.9	700	176.49	3.97	9820
20.4	800	226.44	3.53	10425
25.6	900	284.16	3.17	11027
30.2	1050	335.22	3.13	12025
"""

tableAPC7060_11 = """
1.7	100	18.87	5.30	4957
3.7	200	41.07	4.87	6990
6.3	300	69.93	4.29	8444
8.8	400	97.68	4.10	9731
12	500	133.2	3.75	10895
15.9	600	176.49	3.40	11826
21.3	750	236.43	3.17	13244
"""

tableAPC7060_15 = """
1.3	100	19.24	5.20	4932
2.8	200	41.44	4.83	6995
4.7	300	69.56	4.31	8481
6.7	400	99.16	4.03	9772
9	500	133.2	3.75	10878
11.8	600	174.64	3.44	11848
14.6	700	216.08	3.24	12964
18.3	800	270.84	2.95	13774
22.2	900	328.56	2.74	14584
25.9	1000	383.32	2.61	15238
30.6	1100	452.88	2.43	16371
34.4	1210	509.12	2.38	16243
"""

def table2DF(table):
    table = np.asarray([[float(subval) for subval in val.split("\t")] for val in table.split('\n') if val != ""])
    df = pd.DataFrame({
        "Amps" : table[:,0],
        "Thrust" : table[:,1]*9.80665*1e-3,
        "Watts" : table[:,2],
        "Efficiency" : table[:,3],
        "RPM" : table[:,4],
    })
    
    return df

tableAPC9045 = table2DF(tableAPC9045)
tableAPC9047 = table2DF(tableAPC9047)
tableAPC8038 = table2DF(tableAPC8038)
tableAPC8060 = table2DF(tableAPC8060)
tableAPC7060_11 = table2DF(tableAPC7060_11)
tableAPC7060_15 = table2DF(tableAPC7060_15)

X2212V3KV1400 = {
    11.1: {
        "APC9045" : tableAPC9045,
        "APC9047" : tableAPC9047,
        "APC8038" : tableAPC8038,
        "APC8060" : tableAPC8060,
        "APC7060" : tableAPC7060_11
    },
    14.8: {
        "APC7060" : tableAPC7060_15
    }
}

In [491]:
engineDB = {
    "X2212V3_KV980"  : X2212V3KV980,
    "X2212V3_KV1250" : X2212V3KV1250,
    "X2212V3_KV1400" : X2212V3KV1400
}

In [52]:
def interpMotorProps_Thrust(T, voltage=11.1, model="X2212V3_KV980", prop="APC9045"):
    A = np.interp([T],engineDB[model][voltage][prop]["Thrust"],engineDB[model][voltage][prop]["Amps"])[0]
    RPM = np.interp([T],engineDB[model][voltage][prop]["Thrust"],engineDB[model][voltage][prop]["RPM"])[0]
    W = np.interp([T],engineDB[model][voltage][prop]["Thrust"],engineDB[model][voltage][prop]["Watts"])[0]
    
    return dict(Amps=A, RPM=RPM, Watts=W)

Power estimate:
* Raspberry Pi: 5V/0.6-1.25A
* Navio2: 4.75-5.25V/0.15A

Mass estimate:
* Raspberry Pi: 0.045 kg
* Navio2: 0.023 kg

In [443]:
# Hover time [s]
HT = 15*60
# Cells in series
S = 3
# Payload mass
m_PL = 1.4987915556525386
# Thrust-to-weight ratio
TWR = 2
# Avionics-to-motor power ratio
AMR = .1
# Voltage
V = 11.1
# Power safety factor
PSF = 1.1

T_prop = (m_PL*9.80665*TWR/4)/np.sqrt(2)
P_prop = interpMotorProps_Thrust(T_prop, voltage=11.1, prop="APC1047")["Watts"]
I_prop = interpMotorProps_Thrust(T_prop, voltage=11.1, prop="APC1047")["Amps"]
P_prop_max = interpMotorProps_Thrust(T_prop*np.sqrt(2), voltage=11.1, prop="APC1047")["Watts"]
I_prop_max = interpMotorProps_Thrust(T_prop*np.sqrt(2), voltage=11.1, prop="APC1047")["Amps"]
AMR = 5.1*1.15/P_prop

P_PL = ((1+AMR/4)*P_prop*4)*PSF
Ah = P_PL*(HT/3600)/V

In [454]:
uav.rotorradius = ((10*2.54)/2)*1e-2

In [455]:
# Est. tot. battery mass
m_B_est = uav.batterymass(capacity=Ah*1e3, seriesnumber=S)*1e-3

In [456]:
# Est. motor mass
m_BL_est = uav.motormassBL(Kv=980, P=P_prop, I=I_prop)*1e-2

In [457]:
# Actual motor mass
m_BL_act = 57*1e-3

In [458]:
# Est. engine speed controller mass
m_ESC_est = uav.ESCmassBL(Imax=I_prop_max)*1e-3

In [459]:
# Est. rotor mass
m_R_est = uav.rotormass()*1e-3

In [460]:
# Actual rotor mass
m_R_act = 0.42/35.274

In [461]:
m_A_est = uav.airframemass(R=uav.rotorradius, capacity=Ah*1e3, seriesnumber=S)*1e-3

In [462]:
m_PL_est = m_B_est + 4*(m_BL_est+m_ESC_est+m_R_est) + m_A_est

In [463]:
m_PL_est

2.1595580514993626

In [486]:
engineDB["X2212V3_KV980"][11.1]["APC1047"]

,Amps,Thrust,Watts,Efficiency,RPM
0,0.70,0.980665,7.77,12.87,2612.0
1,1.70,1.961330,18.87,10.60,3605.0
2,3.00,2.941995,33.30,9.01,4264.0
3,4.50,3.922660,49.95,8.01,4887.0
4,6.40,4.903325,71.04,7.04,5418.0
5,8.30,5.883990,92.13,6.51,5872.0
6,10.45,6.864655,115.99,6.03,6335.0
7,12.70,7.845320,5.00,5.67,6983.0
8,15.30,8.825985,140.97,5.30,7378.0
9,19.90,10.787315,169.83,4.98,7748.0


In [492]:
def PL_est(m_PL = 0.5, HT=15*60, TWR=1.3, Kv=980, V=11.1, model="X2212V3_KV980", prop="APC1047"):
    uav = Winslow()
    
    # Hover time [s]
    # HT = 15*60
    # Cells in series
    S = 3
    # Payload mass
    # m_PL = 0.5
    # Thrust-to-weight ratio
    # TWR = 1.3
    # Avionics-to-motor power ratio
    AMR = .1
    # Voltage
    V = 11.1
    # Power safety factor
    PSF = 1.1

    T_prop = (m_PL*9.80665*TWR/4)/np.sqrt(2)
    P_prop = interpMotorProps_Thrust(T_prop, voltage=V, model=model, prop=prop)["Watts"]
    I_prop = interpMotorProps_Thrust(T_prop, voltage=11.1, model=model, prop=prop)["Amps"]
    P_prop_max = interpMotorProps_Thrust(T_prop*np.sqrt(2), voltage=V, model=model, prop=prop)["Watts"]
    I_prop_max = interpMotorProps_Thrust(T_prop*np.sqrt(2), voltage=V, model=model, prop=prop)["Amps"]
    AMR = 5.1*1.15/P_prop

    P_PL = ((1+AMR/4)*P_prop*4)*PSF
    Ah = P_PL*(HT/3600)/V
    
    uav.rotorradius = ((10*2.54)/2)*1e-2
    
    # Est. tot. battery mass
    m_B_est = uav.batterymass(capacity=Ah*1e3, seriesnumber=S)*1e-3
    
    # Est. motor mass
    m_BL_est = uav.motormassBL(Kv=Kv, P=P_prop, I=I_prop)*1e-2
    
    # Actual motor mass
    m_BL_act = 57*1e-3
    
    # Est. engine speed controller mass
    m_ESC_est = uav.ESCmassBL(Imax=I_prop_max)*1e-3
    
    # Est. rotor mass
    m_R_est = uav.rotormass()*1e-3
    
    # Actual rotor mass
    m_R_act = 0.42/35.274
    
    m_A_est = uav.airframemass(R=uav.rotorradius, capacity=Ah*1e3, seriesnumber=S)*1e-3
    
#     m_PL_est = m_B_est + 4*(m_BL_est+m_ESC_est+m_R_est) + m_A_est
    m_PL_est = m_B_est + 4*(m_BL_act+m_ESC_est+m_R_act) + m_A_est
    
    return m_PL_est

### Preliminary estimate (single motor model tested)

In [504]:
converged = False
mPL = 1e-3
while not converged:
    mPLnew = PL_est(mPL, HT=15*60, TWR=2, model="X2212V3_KV980", prop="APC1038", V=11.1)
    
    if mPLnew <= mPL:
        converged = True
#         mPL = mPLnew
    else:
#         print(mPLnew)
        mPL = mPLnew
    
print(mPL)

1.4967294944831258


In [509]:
uav = Winslow()
    
# Hover time [s]
HT = 15*60
# Cells in series
S = 3
# Payload mass
# m_PL = 2.329326353048053
m_PL = 1.4967294944831258
# Thrust-to-weight ratio
TWR = 2
# Avionics-to-motor power ratio
AMR = .1
# Voltage
V = 11.1
# Power safety factor
PSF = 1.1

T_prop = (m_PL*9.80665*TWR/4)/np.sqrt(2)
P_prop = interpMotorProps_Thrust(T_prop, voltage=11.1, prop="APC1038")["Watts"]
I_prop = interpMotorProps_Thrust(T_prop, voltage=11.1, prop="APC1038")["Amps"]
P_prop_max = interpMotorProps_Thrust(T_prop*np.sqrt(2), voltage=11.1, prop="APC1038")["Watts"]
I_prop_max = interpMotorProps_Thrust(T_prop*np.sqrt(2), voltage=11.1, prop="APC1038")["Amps"]
AMR = 5.1*1.15/P_prop

P_PL = ((1+AMR/4)*P_prop*4)*PSF
Ah = P_PL*(HT/3600)/V

uav.rotorradius = ((10*2.54)/2)*1e-2

# Est. tot. battery mass
m_B_est = uav.batterymass(capacity=Ah*1e3, seriesnumber=S)*1e-3

# Est. motor mass
m_BL_est = uav.motormassBL(Kv=980, P=P_prop, I=I_prop)*1e-2

# Actual motor mass
m_BL_act = 57*1e-3

# Est. engine speed controller mass
m_ESC_est = uav.ESCmassBL(Imax=I_prop_max)*1e-3

# Est. rotor mass
m_R_est = uav.rotormass()*1e-3

# Actual rotor mass
m_R_act = 0.42/35.274

m_A_est = uav.airframemass(R=uav.rotorradius, capacity=Ah*1e3, seriesnumber=S)*1e-3

# m_PL_est = m_B_est + 4*(m_BL_est+m_ESC_est+m_R_est) + m_A_est
m_PL_est = m_B_est + 4*(m_BL_act+m_ESC_est+m_R_act) + m_A_est

In [530]:
configUAV = pd.DataFrame({
    "Airframe mass" : [m_A_est],
    "Payload mass" : [m_PL_est],
    "Rotor mass" : [m_R_act],
    "Motor mass" : [m_BL_act],
    "Battery mass" : [m_B_est],
    "ESC mass" : [m_ESC_est],
    "Rotor radius" : [uav.rotorradius*1e-2],
    "Power tot." : [P_PL],
    "Current max." : [I_prop_max],
    "Power max." : [((1+AMR/4)*P_prop_max*4)*PSF],
    "TWR" : TWR,
    "Hover time" : HT,
    "AMR" : AMR,
    "Battery capacity" : Ah*1e3,
    "Single cell capacity" : (Ah/4)*1e3,
    "Rotor model" : "APC1038",
    "Motor model" : "X2212V3",
    "Motor Kv" : 980
})

print(configUAV.reindex(["Airframe mass", "Battery mass", "ESC mass", "Motor mass", "Rotor mass", "Motor Kv", "Motor model", "Rotor model", "Payload mass", "Current max.", "Power max.", "Power tot.", "Battery capacity", "Single cell capacity", "Hover time", "AMR", "TWR"], axis=1).transpose().to_latex())

\begin{tabular}{ll}
\toprule
{} &           0 \\
\midrule
Airframe mass        &    0.618016 \\
Battery mass         &    0.569293 \\
ESC mass             &  0.00844846 \\
Motor mass           &       0.057 \\
Rotor mass           &   0.0119068 \\
Motor Kv             &         980 \\
Motor model          &     X2212V3 \\
Rotor model          &     APC1038 \\
Payload mass         &     1.49673 \\
Current max.         &      11.264 \\
Power max.           &     560.782 \\
Power tot.           &     339.791 \\
Battery capacity     &     7652.95 \\
Single cell capacity &     1913.24 \\
Hover time           &         900 \\
AMR                  &   0.0774166 \\
TWR                  &           2 \\
\bottomrule
\end{tabular}

